# Combining EnergyPlus and EnergyHub Evaluators
This notebook covers different ways to use EnergyPlus and PyEHub Evaluators together.

In [ ]:
import numpy as np
import pandas as pd
from besos import eppy_funcs as ef, pyehub_funcs as pf
from besos.evaluator import EvaluatorEH, EvaluatorEP
from besos.objectives import MeterReader
from besos.parameters import (
    FieldSelector,
    Parameter,
    PathSelector,
    RangeParameter,
)
from besos.problem import EHProblem, EPProblem

### Custom EnergyPlus Evaluator Functions
First we need to define a way to get a whole time series from EnergyPlus Evaluator, not just an objective function value.
To extract a time series from an EnergyPlus Evaluator the default summation function must be replaced.
The function `timeseriesfunc` returns the entire Pandas Series from the output of the EnergyPlus simulation.

In [ ]:
def timeseriesfunc(result):
    return result.data["Value"]

### Create an EnergyPlus Evaluator
Here is a standard EnergyPlus Evaluator for editing the lighting power density for the default building and getting the electricity demand time series.
The output is a Pandas Series, and the units are Joules.

In [ ]:
building = ef.get_building()
EPparameters = [
    Parameter(
        FieldSelector("Lights", "*", "Watts per Zone Floor Area"),
        value_descriptor=RangeParameter(8, 12),
        name="Lights Watts/Area",
    )
]
EPobjectives = MeterReader("Electricity:Facility", func=timeseriesfunc)
problem = EPProblem(EPparameters, EPobjectives)
evaluator = EvaluatorEP(problem, building)
result = evaluator([8])
result

### EnergyPlus Evaluator Output conversions
To ensure the output of the EnergyPlus evaluator is in the correct format for the Energy Hub, some conversions are required.

First the result is converted from a Pandas Series to a dataframe.

In [ ]:
act_result = result[0].to_frame()

#### Splitting into days
Then because EnergyPlus simulated a summer design day and a winter design day, the output is split and their indexes reset.

In [ ]:
cold_result = act_result.head(24)
cold_result = cold_result.reset_index()
warm_result = act_result.tail(24)
warm_result = warm_result.reset_index()

#### Unit Conversions
The output for an energy `Output:Meter` in EnergyPlus is in Joules but EnergyHub deals with kWh so the entire dataframe for both days is converted.
They are then turned into dictionaries with the keys being the time series index.

In [ ]:
cold_result = cold_result / 3600000
cold_dict = cold_result.to_dict()
cold_dict = cold_dict["Value"]

warm_result = warm_result / 3600000
warm_dict = warm_result.to_dict()
warm_dict = warm_dict["Value"]

#### Wrapping in Dictionaries
Lastly the dictionaries are wrapped as lists to match the input format for EnergyHub Evaluators.

In [ ]:
cold_input = [cold_dict]
warm_input = [warm_dict]

### Create a PyEHub Evaluator
Here is a standard PyEHub Evaluator for editing the electrical load of a simple energy hub, minimizing the total cost and outputting both the total cost and total carbon emissions from the optimizied hub.
See [EHEvaluator](EHEvaluator.ipynb) for more details.
It is applied to the Energy Hub model specified in `config.yaml`.

In [ ]:
EHparameters = [Parameter(PathSelector(["LOADS", "Elec"]))]
EHobjectives = ["total_cost", "total_carbon"]
EHproblem = EHProblem(EHparameters, EHobjectives)
hub = pf.get_hub()
EHevaluator = EvaluatorEH(EHproblem, hub)

#### Single timeseries for PyEHub Evaluator
The wrapped dictionary inputs can be used directly as input for the PyEHub Evaluator.

In [ ]:
result1 = EHevaluator(cold_input)
result1

In [ ]:
result2 = EHevaluator(warm_input)
result2

#### Dataframe of time series
These inputs can be combined into a single dataframe and used as input for the evaluators.

In [ ]:
seasons_df = pd.DataFrame(np.array([warm_input, cold_input]), columns=["p1"])

In [ ]:
result3 = EHevaluator.df_apply(seasons_df)
result3

### Energy Plus Dataframe input
If the input of the EnergyPlus Evaluator is a dataframe, then `df_apply` can be used to execute the evaluator.

In [ ]:
EPdf = pd.DataFrame(np.array([[8], [9], [10], [12]]), columns=["p1"])
df_results = evaluator.df_apply(EPdf)
df_results

#### EnergyPlus Dataframe output conversion
The conversions must be done while maintaining a dataframe to be used with `df_apply` for the PyEHub Evaluator.
The previous splitting of days, unit conversions, and wrapping dictionaries must be done for every output of the EnergyPlus Dataframe, then appended into a larger dataframe to be used as the input to the Energy Hub.
The larger dataframe is prepared with the number of objectives from the EnergyPlus simulator.

In [ ]:
results_dicts = df_results.to_dict()
columnnames = []
for j in results_dicts:
    columnnames.append(j)

df_input = pd.DataFrame(columns=columnnames)

for j in results_dicts:
    for i in results_dicts[j]:

        act_result = results_dicts[j][i].to_frame()

        cold_result = act_result.head(24)
        cold_result = cold_result.reset_index()
        warm_result = act_result.tail(24)
        warm_result = warm_result.reset_index()

        cold_result = cold_result / 3600000
        cold_dict = cold_result.to_dict()
        cold_dict = cold_dict["Value"]

        warm_result = warm_result / 3600000
        warm_dict = warm_result.to_dict()
        warm_dict = warm_dict["Value"]

        temp_df1 = pd.DataFrame(np.array(cold_input), columns=[j])
        temp_df2 = pd.DataFrame(np.array(warm_input), columns=[j])
        df_input = df_input.append(temp_df1, ignore_index=True)
        df_input = df_input.append(temp_df2, ignore_index=True)

In [ ]:
df_input

### EnergyPlus to EnergyHub Dataframe output
The converted output from the EnergyPlus Evaluator can then be used as input for the PyEHub evaluator.

In [ ]:
result4 = EHevaluator.df_apply(df_input)
result4